<a href="https://colab.research.google.com/github/ilman79/Machine-Learning/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from flask import Flask, request, jsonify
import torch
from PIL import Image
from torchvision.transforms import functional as F
from yolov5.models.experimental import attempt_load
from yolov5.utils.general import non_max_suppression, scale_coords

ModuleNotFoundError: ignored

In [10]:
app = Flask(__name__)

In [11]:
@app.route('/detect', methods=['POST'])
def detect():
    # Memeriksa apakah ada file gambar dalam permintaan
    if 'image' not in request.files:
        return jsonify({'error': 'No image found.'}), 400

    # Memuat gambar dari permintaan
    image = Image.open(request.files['image']).convert('RGB')

    # Mengubah gambar menjadi tensor
    image_tensor = F.to_tensor(image)

    # Menambahkan dimensi batch pada tensor gambar
    image_tensor = image_tensor.unsqueeze(0)

    # Memeriksa apakah ada GPU yang tersedia
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Memuat model YOLOv5
    model = attempt_load('best.pt', map_location=device)

    # Menggunakan model untuk mendeteksi objek pada gambar
    detections = model(image_tensor.to(device))[0]

    # Menerapkan non-maximum suppression untuk membatasi hasil deteksi
    detections = non_max_suppression(detections, conf_thres=0.3, iou_thres=0.45)

    # Mengubah koordinat deteksi ke dalam skala gambar asli
    scale_coords(image.size, detections[0][:, :4], image.size)

    # Membuat daftar hasil deteksi dalam format yang dapat dijsonifikasi
    results = []
    for x1, y1, x2, y2, conf, cls in detections[0]:
        results.append({
            'class': int(cls),
            'confidence': float(conf),
            'bounding_box': {
                'x1': float(x1),
                'y1': float(y1),
                'x2': float(x2),
                'y2': float(y2)
            }
        })

    # Mengembalikan hasil deteksi dalam format JSON
    return jsonify({'detections': results}), 200


In [6]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [8]:
import requests

# Menentukan URL API
url = 'http://localhost:5000/detect'

# Membuka file gambar yang akan dideteksi
image_file = open('image.jpg', 'rb')

# Membuat permintaan POST ke API dengan file gambar
response = requests.post(url, files={'image': image_file})

# Mengecek kode status respon
if response.status_code == 200:
    # Mengambil hasil deteksi dari respon JSON
    detections = response.json()['detections']
    
    # Menampilkan hasil deteksi
    for detection in detections:
        class_name = detection['class']
        confidence = detection['confidence']
        bounding_box = detection['bounding_box']
        
        print('Class:', class_name)
        print('Confidence:', confidence)
        print('Bounding Box:', bounding_box)
        print('---')
else:
    print('Error:', response.text)


FileNotFoundError: ignored